In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')

In [ ]:
# downcast the dataset
def downcast_dtype(df):
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [ ]:
train_df.info()

In [ ]:
train_df = downcast_dtype(train_df)

In [ ]:
train_df.info()

**Creating Dataset**

In [ ]:
train_df.head()

In [ ]:
# transform (橫轉直)
train_df = train_df.T

In [ ]:
train_df.head(10)

In [ ]:
# 只要d_1, d_2, ...
train_df = train_df[6:]

In [ ]:
# scale the dataset becomes 0 to 1
sc = MinMaxScaler(feature_range=(0, 1))
train_df = sc.fit_transform(train_df)

In [ ]:
print(train_df)

In [ ]:
# 取14日, 預測第15日
timesteps = 14

x_train = []
y_train = []

for i in range(timesteps, 1913):
    x_train.append(train_df[i-timesteps:i])
    y_train.append(train_df[i][0:30490])

In [ ]:
x_train = np.array(x_train, dtype='float16')
y_train = np.array(y_train, dtype='float16')

In [ ]:
x_train

In [ ]:
print(f'x_train.shape: {x_train.shape}')
print(f'y_train.shape: {y_train.shape}')

In [ ]:
# model
model = tf.keras.Sequential()

# 1 LSTM layer, 1 Dense layer
model.add(tf.keras.layers.LSTM(
            units = 64,
            input_shape = (np.array(x_train).shape[1], np.array(x_train).shape[2])
))
model.add(tf.keras.layers.Dense(30490))

model.compile(
    loss = 'mean_squared_error',
    optimizer = tf.keras.optimizers.Adam(0.001)
)
model.summary()

In [ ]:
model.fit(
    x_train,
    y_train,
    epochs = 10,
    batch_size = 10
)

In [ ]:
# test

In [ ]:
inputs = train_df[-timesteps:]
inputs = sc.transform(inputs)

In [ ]:
inputs

In [ ]:
x_test = []
x_test.append(inputs[0:timesteps])
x_test = np.array(x_test)

In [ ]:
x_test

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predictions

In [ ]:
# in order to predict more days, we can use for-loop to implement. But in here, I am not planning to do it.